<a href="https://colab.research.google.com/github/shyookml/fdl/blob/Chap.4/mnist_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import os

# 데이터 불러오기
(x_train_org,y_train_org),(x_test,y_test)=load_data(path='mnist.npz')

# 트레인 데이터를 진짜 train 할 것과 검증할 테스느 데이터로 나누기
x_train,x_val,y_train,y_val=train_test_split(x_train_org,y_train_org,test_size=0.3,random_state=777)

num_x_train=x_train.shape[0]
num_x_val=x_val.shape[0]
num_x_test=x_test.shape[0]

# 1차원 데이터로 전처리
x_train=(x_train.reshape((num_x_train,28*28)))/255
x_val=(x_val.reshape((num_x_val,28*28)))/255
x_test=(x_test.reshape((num_x_test,28*28)))/255

# categorize 위한 train data 변환
# 주의: 변수 이름 바꿨음!!!
y_train_categ=to_categorical(y_train)
y_val_categ=to_categorical(y_val)
y_test_categ=to_categorical(y_test)

# sepcial statements for TPU
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
  # 모델 만들기: 3층 DNN
  model=Sequential()

  model.add(Dense(64,activation='relu', input_shape=(784,)))
  model.add(Dense(32,activation='relu'))
  model.add(Dense(10,activation='softmax'))

  # 모델 묶고
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
  # 학습
  history=model.fit(x_train,y_train_categ,epochs=30,batch_size=128,validation_data=(x_val,y_val_categ))

model.evaluate(x_test,y_test_categ)

# 학습된 모델을 가지고 test 데이터의 값 예측 테스트
import numpy as np

results=model.predict(x_test)

print(results.shape)
np.set_printoptions(precision=7)
print(f'각 클래스에 속할 확률\n{results[0]}')
print(f"estimation:{np.argmax(results[0])}")
print(f"정답:{y_test[0]}")

Epoch 1/30
329/329 [==============================] - 13s 30ms/step - loss: 0.4906 - acc: 0.8612 - val_loss: 0.2584 - val_acc: 0.9248
Epoch 2/30
329/329 [==============================] - 8s 23ms/step - loss: 0.2130 - acc: 0.9387 - val_loss: 0.1919 - val_acc: 0.9453
Epoch 3/30
329/329 [==============================] - 8s 24ms/step - loss: 0.1613 - acc: 0.9525 - val_loss: 0.1728 - val_acc: 0.9504
Epoch 4/30
329/329 [==============================] - 9s 27ms/step - loss: 0.1334 - acc: 0.9610 - val_loss: 0.1482 - val_acc: 0.9568
Epoch 5/30
329/329 [==============================] - 9s 27ms/step - loss: 0.1142 - acc: 0.9671 - val_loss: 0.1373 - val_acc: 0.9594
Epoch 6/30
329/329 [==============================] - 8s 24ms/step - loss: 0.0965 - acc: 0.9718 - val_loss: 0.1232 - val_acc: 0.9629
Epoch 7/30
329/329 [==============================] - 8s 25ms/step - loss: 0.0848 - acc: 0.9749 - val_loss: 0.1196 - val_acc: 0.9660
Epoch 8/30
329/329 [==============================] - 8s 25ms/step -